In [25]:
import pandas as pd
import requests
import re

In [26]:
COMPANIES_FILENAME = 'top_ten.csv'
PAGESPEED_BASE_URL = 'https://www.googleapis.com/pagespeedonline/v5/runPagespeed'
with open('PAGESPEED_API_KEY') as key_file:
  PAGESPEED_API_KEY = key_file.read().strip() # bad style? should be constant per run
AUDIT_KEYS = [
  'first-contentful-paint',
  'speed-index',
  'interactive',
  'first-meaningful-paint',
  'first-cpu-idle',
  'estimated-input-latency'
]

In [27]:
companies = pd.read_csv(COMPANIES_FILENAME)
companies = companies.drop('ticker_symbol', axis=1)
sample_companies = companies[:2]
sample_companies

,name,web_url
0,Microsoft Corp,https://www.microsoft.com/en-us/
1,Apple Inc.,https://www.apple.com/


In [28]:
def pagespeed_query_url(page):
    return f'{PAGESPEED_BASE_URL}?url={page}&key={PAGESPEED_API_KEY}'


In [29]:
[pagespeed_query_url(page) for page in companies[:3]['web_url']];

In [160]:
resp = requests.get(url=PAGESPEED_BASE_URL, params={'url': page, 'key': PAGESPEED_API_KEY})

[API doc](https://developers.google.com/speed/docs/insights/v5/reference/pagespeedapi/runpagespeed#response)

In [164]:
def unit(display_value):
    return re.search('\w+$', display_value)[0]

def verify_units(audits):
    for key in AUDIT_KEYS:
        if key == 'estimated-input-latency':
            if unit(audits[key]['displayValue']) != 'ms':
                raise BaseException('expected to always receive milliseconds')
        else:
            if unit(audits[key]['displayValue']) != 's':
                raise BaseException('expected to always receive seconds')
            

In [177]:
def displayToFloat(displayValue):
    return float(re.search('^\d+\.?\d*', displayValue)[0])

In [166]:
def audits(response):
    audits = response.json()['lighthouseResult']['audits']
    verify_units(audits)
    return {
        audit_key : displayToFloat(audits[audit_key]['displayValue'])
        for audit_key in AUDIT_KEYS
    }

In [178]:
audits(resp)

{'first-contentful-paint': 0.6,
 'speed-index': 3.0,
 'interactive': 0.6,
 'first-meaningful-paint': 0.6,
 'first-cpu-idle': 0.6,
 'estimated-input-latency': 10.0}